# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.839      0.382      0.677        340       5716
author                    0.805      0.528      0.729        509       5885
capital                   0.651      0.295      0.524         95       5471
contains                  0.780      0.597      0.735       3904       9280
film_performance          0.760      0.568      0.712        766       6142
founders                  0.844      0.371      0.673        380       5756
genre                     0.549      0.165      0.374        170       5546
has_sibling               0.904      0.246      0.590        499       5875
has_spouse                0.894      0.327      0.663        594       5970
is_a                      0.720      0.237      0.512        497       5873
nationality               0.565      0.159      0.374        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.575 Córdoba
     2.512 Taluks
     2.447 Valais
     ..... .....
    -1.195 Asia
    -1.208 Afghanistan
    -1.420 Cook

Highest and lowest feature weights for relation author:

     2.860 wrote
     2.553 author
     2.345 by
     ..... .....
    -2.012 or
    -2.234 directed
    -7.832 dystopian

Highest and lowest feature weights for relation capital:

     4.015 capital
     2.281 city
     1.747 km
     ..... .....
    -1.148 coast
    -1.351 County
    -1.478 Cook

Highest and lowest feature weights for relation contains:

     2.861 third-largest
     2.466 bordered
     2.123 tiny
     ..... .....
    -2.425 who
    -3.456 Midlands
    -3.784 Antrim

Highest and lowest feature weights for relation film_performance:

     3.831 starring
     3.675 alongside
     3.453 co-starring
     ..... .....
    -1.998 Westminster
    -2.346 Malice
    -2.361 Wonderland

Highest and lowest feature weights for relation founders

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [14]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [15]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [16]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [17]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.852      0.456      0.726        340       5716
author                    0.807      0.436      0.690        509       5885
capital                   0.535      0.242      0.431         95       5471
contains                  0.650      0.415      0.584       3904       9280
film_performance          0.803      0.315      0.613        766       6142
founders                  0.821      0.242      0.556        380       5756
genre                     0.357      0.059      0.177        170       5546
has_sibling               0.858      0.242      0.569        499       5875
has_spouse                0.888      0.347      0.677        594       5970
is_a                      0.690      0.157      0.411        497       5873
nationality               0.648      0.196      0.444        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [18]:
def run_svm_model_factory():
    from sklearn.svm import SVC
    svm_results = rel_ext.experiment(
                        splits,
                        train_split='train',
                        test_split='dev',
                        featurizers=featurizers,
                        model_factory=lambda: SVC(kernel='linear'),
                        verbose=True)
    return svm_results



In [19]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [20]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.810      0.350      0.641        340       5716
author                    0.773      0.601      0.731        509       5885
capital                   0.646      0.326      0.540         95       5471
contains                  0.767      0.598      0.726       3904       9280
film_performance          0.749      0.619      0.719        766       6142
founders                  0.738      0.429      0.645        380       5756
genre                     0.581      0.253      0.461        170       5546
has_sibling               0.865      0.244      0.574        499       5875
has_spouse                0.850      0.343      0.656        594       5970
is_a                      0.650      0.280      0.514        497       5873
nationality               0.526      0.199      0.396        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [21]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE

directional_bog_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[directional_bag_of_words_featurizer],
    model_factory=model_factory,
    verbose=True)

print("Number of feature names: {}".format(len(directional_bog_results['vectorizer'].get_feature_names())))


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.864      0.412      0.709        340       5716
author                    0.830      0.583      0.765        509       5885
capital                   0.703      0.274      0.535         95       5471
contains                  0.810      0.640      0.769       3904       9280
film_performance          0.820      0.662      0.783        766       6142
founders                  0.797      0.392      0.660        380       5756
genre                     0.678      0.235      0.493        170       5546
has_sibling               0.894      0.253      0.593        499       5875
has_spouse                0.888      0.347      0.677        594       5970
is_a                      0.707      0.247      0.516        497       5873
nationality               0.653      0.206      0.455        301       5677
parents     

In [22]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [23]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [24]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.middle_POS)
        bigrams = get_tag_bigrams(tags)
        for pos in bigrams:
            feature_counter[pos] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.middle_POS)
        bigrams = get_tag_bigrams(tags)
        for pos in bigrams:
            feature_counter[pos] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    bigram_POS = []
    if len(s) == 0:
        return bigram_POS
    
    previous = start_symbol
    for tag in s:
        bigram_POS.append(previous + ' ' + tag)
        previous = tag
    bigram_POS.append(s[-1] + ' ' + end_symbol)
    
    return bigram_POS


def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
middle_bigram_pos_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_bigram_pos_tag_featurizer],
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.836      0.359      0.660        340       5716
author                    0.720      0.318      0.575        509       5885
capital                   0.640      0.168      0.410         95       5471
contains                  0.750      0.596      0.713       3904       9280
film_performance          0.700      0.443      0.627        766       6142
founders                  0.573      0.166      0.384        380       5756
genre                     0.683      0.165      0.419        170       5546
has_sibling               0.694      0.172      0.432        499       5875
has_spouse                0.777      0.258      0.554        594       5970
is_a                      0.626      0.145      0.376        497       5873
nationality               0.422      0.063      0.198        301       5677
parents     

In [25]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [26]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [27]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        synsets = get_synsets(ex.middle_POS)
        for synset in synsets:
            feature_counter[synset] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        synsets = get_synsets(ex.middle_POS)
        for synset in synsets:
            feature_counter[synset] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    synsets = []
    for word, pos in wt:
        for synset in wn.synsets(word, pos=convert_tag(pos)):
            synsets.append(str(synset))
    
    return synsets


def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE

synset_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer],
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.807      0.344      0.636        340       5716
author                    0.825      0.434      0.699        509       5885
capital                   0.558      0.305      0.479         95       5471
contains                  0.770      0.592      0.726       3904       9280
film_performance          0.777      0.550      0.717        766       6142
founders                  0.811      0.374      0.657        380       5756
genre                     0.468      0.212      0.377        170       5546
has_sibling               0.863      0.214      0.538        499       5875
has_spouse                0.843      0.306      0.624        594       5970
is_a                      0.634      0.237      0.475        497       5873
nationality               0.494      0.130      0.316        301       5677
parents     

In [28]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [29]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [30]:
'''
Helper functions for the original system. Run this cell before evaluating the orignal system.
'''

def dir_left_bag_of_words_featurizer(kbt, corpus, feature_counter):
    prefix = "left_"
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.left.split(' '):
            feature_counter[prefix + word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.left.split(' '):
            feature_counter[prefix + word + object_subject_suffix] += 1

    return feature_counter

def dir_right_bag_of_words_featurizer(kbt, corpus, feature_counter):
    prefix = "right_"
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.right.split(' '):
            feature_counter[prefix + word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.right.split(' '):
            feature_counter[prefix + word + object_subject_suffix] += 1

    return feature_counter

def middle_length_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        feature_counter[ex.middle] += len(ex.middle.split(' '))
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter[ex.middle] += len(ex.middle.split(' '))
    
    return feature_counter

def middle_trigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        tags = get_tags(ex.middle_POS)
        trigrams = get_tag_trigrams(tags)
        for pos in trigrams:
            feature_counter[pos] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        tags = get_tags(ex.middle_POS)
        trigrams = get_tag_trigrams(tags)
        for pos in trigrams:
            feature_counter[pos] += 1

    return feature_counter


def get_tag_trigrams(s):
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    trigram_POS = []
    if len(s) < 2:
        return trigram_POS
    
    previous = start_symbol
    for tag0,tag1 in zip(s[:-1], s[1:]):
        trigram_POS.append(previous + ' ' + tag0 + ' ' + tag1)
        previous = tag0
    trigram_POS.append(s[-2] + ' ' + s[-1] + ' ' + end_symbol)
    
    return trigram_POS


def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)



In [31]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM 
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My system: lots of hand built features, as suggested in the hint: 
#       simple_bag_of_words_featurizer, 
#       directional_bag_of_words_featurizer, 
#       middle_bigram_pos_tag_featurizer,
#       middle_trigram_pos_tag_featurizer,
#       middle_length_featurizer,
#       dir_left_bag_of_words_featurizer,
#       dir_right_bag_of_words_featurizer,
#       synset_featurizer
# I also tried other ML models such as SVM with different kernels, which did not 
# help much with the default hyperparameters
# My peak score was: 0.729
if 'IS_GRADESCOPE_ENV' not in os.environ:
    my_featurizers = [simple_bag_of_words_featurizer, 
                      directional_bag_of_words_featurizer, 
                      middle_bigram_pos_tag_featurizer,
                      middle_trigram_pos_tag_featurizer,
                      middle_length_featurizer,
                      dir_left_bag_of_words_featurizer,
                      dir_right_bag_of_words_featurizer,
                      synset_featurizer]

    my_results = rel_ext.experiment(
                            splits,
                            train_split='train',
                            test_split='dev',
                            featurizers=my_featurizers,
                            model_factory=model_factory,
                            verbose=True)

# STOP COMMENT: Please do not remove this comment.

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.911      0.453      0.758        340       5716
author                    0.839      0.770      0.825        509       5885
capital                   0.639      0.242      0.481         95       5471
contains                  0.850      0.768      0.833       3904       9280
film_performance          0.834      0.721      0.808        766       6142
founders                  0.794      0.529      0.722        380       5756
genre                     0.800      0.376      0.653        170       5546
has_sibling               0.882      0.673      0.830        499       5875
has_spouse                0.796      0.678      0.770        594       5970
is_a                      0.787      0.565      0.730        497       5873
nationality               0.769      0.664      0.746        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [34]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    rel_ext_data_home_test = os.path.join(rel_ext_data_home, 'bakeoff-rel_ext-test-data')
    rel_ext.bake_off_experiment(my_results, rel_ext_data_home_test)




relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.901      0.521      0.786        438       7122
author                    0.873      0.758      0.847        645       7329
capital                   0.615      0.278      0.495        115       6799
contains                  0.831      0.754      0.814       3808      10492
film_performance          0.868      0.740      0.839       1011       7695
founders                  0.773      0.574      0.723        444       7128
genre                     0.786      0.351      0.630        188       6872
has_sibling               0.883      0.676      0.832        717       7401
has_spouse                0.799      0.690      0.775        780       7464
is_a                      0.744      0.527      0.687        611       7295
nationality               0.753      0.587      0.712        383       7067
parents     

In [35]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.724